In [ ]:
cd /content/drive/MyDrive/2022/dacon/구내식당식수인원예측경진대회

In [2]:
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
import io
import pandas as pd
import matplotlib as mpl
import matplotlib.pylab as plt
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rc
import numpy as np
import seaborn as sns
import requests
import calendar
import datetime
from bs4 import BeautifulSoup

In [241]:
data = []
nowyear = 2021
nowmonth = 1

In [276]:
weather = pd.DataFrame(index=range(0,3), columns = {'avg_tmp','max_tmp','min_tmp','rain'})
weather

,max_tmp,min_tmp,rain,avg_tmp
0,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN


#### 관측, 기후

In [314]:
date, avg, max, min, rain = [], [], [], [], []

for year in range(2016, nowyear+1):
  print('y : '+str(year) + '  year : ' + str(nowyear))
  if str(year)==str(nowyear):
    monthrange = int(nowmonth)+1
  else:
    monthrange = 13
  
  for month in range(1, monthrange):
    print(year, month)
    response = requests.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=192&yy='+str(year)+'&mm=' +str(month)+'&obs=1') # 진주
    #response = requests.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=108&yy='+str(year)+'&mm=' +str(month)+'&obs=1') # 서울
    soup = BeautifulSoup(response.content, 'html.parser')
    table = soup.select(".over-scroll > .table-col > tbody > tr > td > span")

    for a in table:
      if a.string:
        if a.string[-1] == '일':
          date.append(str(year)+'-'+str(month)+'-'+str(a.string[:-1]))
        elif a.string.startswith('평균기온'):
          if len(a.string[5:-1]) != 0:
            avg.append(float(a.string[5:-1]))
          else:
            avg.append(None)
        elif a.string.startswith('최고기온'):
          max.append(float(a.string[5:-1]))
        elif a.string.startswith('최저기온'):
          min.append(float(a.string[5:-1]))
        elif a.string.startswith('일강수량'):
          if len(a.string[5:]) == 3:
            rain.append(0)
          else:
            rain.append(a.string[5:-2])

y : 2016  year : 2021
2016 1
2016 2
2016 3
2016 4
2016 5
2016 6
2016 7
2016 8
2016 9
2016 10
2016 11
2016 12
y : 2017  year : 2021
2017 1
2017 2
2017 3
2017 4
2017 5
2017 6
2017 7
2017 8
2017 9
2017 10
2017 11
2017 12
y : 2018  year : 2021
2018 1
2018 2
2018 3
2018 4
2018 5
2018 6
2018 7
2018 8
2018 9
2018 10
2018 11
2018 12
y : 2019  year : 2021
2019 1
2019 2
2019 3
2019 4
2019 5
2019 6
2019 7
2019 8
2019 9
2019 10
2019 11
2019 12
y : 2020  year : 2021
2020 1
2020 2
2020 3
2020 4
2020 5
2020 6
2020 7
2020 8
2020 9
2020 10
2020 11
2020 12
y : 2021  year : 2021
2021 1


In [315]:
print(len(date),len(avg),len(min), len(max), len(rain))

1858 1858 1858 1858 1858


In [316]:
weather = pd.DataFrame(date,columns=['date'])
weather['avg'] = avg
weather['max'] = max
weather['min'] = min
weather['rain'] = rain

In [317]:
weather

,date,avg,max,min,rain
0,2016-1-1,-0.1,8.7,-6.7,0
1,2016-1-2,1.8,10.8,-3.6,0
2,2016-1-3,4.0,14.5,-2.2,0
3,2016-1-4,4.8,15.0,-0.3,0
4,2016-1-5,0.5,4.9,-2.6,0.0
...,...,...,...,...,...
1853,2021-1-27,4.1,11.4,-1.2,0
1854,2021-1-28,1.4,10.7,-3.0,0
1855,2021-1-29,-2.3,3.0,-6.0,0
1856,2021-1-30,-0.3,9.1,-9.2,0


In [318]:
weather.to_csv('weather_jinju.csv')

#### 상대 습도

In [ ]:
response = requests.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-element.do?stn=192&yy=2018&obs=12') # 진주
#response = requests.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn=108&yy='+str(year)+'&mm=' +str(month)+'&obs=1') # 서울
soup = BeautifulSoup(response.content, 'html.parser')
table = soup.select(".over-scroll > .table-col > tbody > tr > td > span")
table[:50]

In [332]:
date, humidity = [], []
for a in table[:50]:
  if a.string:
    if a.string[-1] == '일':
      for i in range(1, 13):
        date.append(str(2016)+'-'+str(i)+'-'+str(a.string[:-1]))
    else:
      humidity.append(a.string)

In [362]:
date, humidity = [], []

for year in range(2016, nowyear+1):
  print('y : '+str(year) + '  year : ' + str(nowyear))
  if str(year)==str(nowyear):
    monthrange = int(nowmonth)+1
  else:
    monthrange = 13
  
  print(year)
  #response = requests.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-element.do?stn=192&yy='+str(year)+'&obs=12') # 진주
  response = requests.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-element.do?stn=108&yy='+str(year)+'&obs=12') # 진주

  soup = BeautifulSoup(response.content, 'html.parser')
  table = soup.select(".over-scroll > .table-col > tbody > tr > td > span")

  for a in table:
    if a.string:
      if a.string == '평균':
        break
      if a.string[-1] == '일':
         for i in range(1, 13):
          date.append(str(year)+'-'+str(i)+'-'+str(a.string[:-1]))
      else:
        humidity.append(a.string)

y : 2016  year : 2021
2016
y : 2017  year : 2021
2017
y : 2018  year : 2021
2018
y : 2019  year : 2021
2019
y : 2020  year : 2021
2020
y : 2021  year : 2021
2021


In [363]:
print(len(date), len(humidity))

2232 2232


In [364]:
humidity_df = pd.DataFrame(date,columns=['date'])
humidity_df['humidity'] = humidity

In [365]:
humidity_df

,date,humidity
0,2016-1-1,73.0
1,2016-2-1,36.0
2,2016-3-1,47.3
3,2016-4-1,26.0
4,2016-5-1,40.5
...,...,...
2227,2021-8-31,93.3
2228,2021-9-31,
2229,2021-10-31,73.3
2230,2021-11-31,


In [366]:
humidity_df.to_csv('humidity_seoul.csv')